In [1]:
from src.utils.bedrock_agent import Agent, SupervisorAgent, Task, region, account_id, agents_helper

boto3 version: 1.36.9


In [2]:
import boto3
import os
import json
import time
import zipfile
import subprocess

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)
iam_client = boto3.client('iam', region)
lambda_client = boto3.client('lambda', region)


# 'anthropic.claude-3-5-sonnet-20240620-v1:0',
# 'anthropic.claude-3-sonnet-20240229-v1:0',
# 'anthropic.claude-3-haiku-20240307-v1:0'

agent_foundation_model = [
    'anthropic.claude-3-5-sonnet-20241022-v2:0'
]




In [3]:
region

'us-west-2'

In [4]:
agent_suffix

'us-west-2-533'

In [5]:
dsl_query_agent_name =  f"dsl-query-agent-{agent_suffix}"
# dsl_query_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{dsl_query_agent_name}'

In [6]:
Agent.set_force_recreate_default(True)

In [7]:

## Schemas formatted for OSS Index
with open('schemas/ecom_shipping_schema.json', 'r') as file:
    ecom_shipping_schema = json.load(file)

ecom_shipping_schema_string= json.dumps(ecom_shipping_schema, indent=2)

In [8]:
ecom_shipping_schema_string

'{\n  "shipping": {\n    "properties": {\n      "order_id": {\n        "type": "keyword"\n      },\n      "tracking_number": {\n        "type": "keyword"\n      },\n      "status": {\n        "type": "keyword"\n      },\n      "created_at": {\n        "type": "date"\n      },\n      "sender": {\n        "properties": {\n          "name": {\n            "type": "text"\n          },\n          "email": {\n            "type": "keyword"\n          },\n          "address": {\n            "type": "text"\n          },\n          "city": {\n            "type": "keyword"\n          },\n          "country": {\n            "type": "keyword"\n          }\n        }\n      },\n      "recipient": {\n        "properties": {\n          "name": {\n            "type": "text"\n          },\n          "email": {\n            "type": "keyword"\n          },\n          "address": {\n            "type": "text"\n          },\n          "city": {\n            "type": "keyword"\n          },\n          "country

In [9]:
dsl_query_agent = Agent.direct_create(
    name=f"dsl-query-agent-{agent_suffix}",
    role="DSL Query Creator",
    goal="Create DSL queries for a given user query",
    instructions=f"""
    You are an expert in generating Query DSL for Elasticsearch-style queries. Your task is to convert a given natural language user question into a well-structured Query DSL.
    
    ## Instructions:
    - Use the provided e-commerce shipping schema to construct the query.
    - Encapsulate the output in <json>...</json> tags.
    - Follow the syntax of the Query DSL strictly; do not introduce elements outside the provided schema.
    
    ## Query Construction Rules:
    - **Keyword fields** (e.g., carrier, status, country): Use `term` for exact matches or `prefix`/`wildcard` for partial matches.
    - **Text fields** (e.g., description, address): Use `match` queries to account for analyzed terms.
    - **Nested fields** (e.g., tracking): Always use `nested` queries.
    - **Date fields**: Use `range` queries with date math for filtering by date ranges.
    - Break down complex queries into smaller parts for accuracy.
    - Think step-by-step before constructing the query.

    ## Schema:
    {ecom_shipping_schema_string}

    ## Output Format:
    - Return only the generated Query DSL within <json>...</json> tags.
    - Do not include explanations, comments, or additional text.
    """,
    tool_code=f"arn:aws:lambda:{region}:{account_id}:function:execute-dsl-query-us-west-2-533",
    tool_defs=[
        {
            "name": "execute_dsl_query",
            "description": "Executes a given DSL query and returns the results",
            "parameters": {
                "dsl_query": {
                    "description": "The DSL query to execute",
                    "type": "string",
                    "required": True,
                }
            }
        }
    ]
)


Deleting existing agent and corresponding lambda for: dsl-query-agent-us-west-2-533...
Found target agent, name: dsl-query-agent-us-west-2-533, id: WJYFKTH1IX
Deleting aliases for agent WJYFKTH1IX...
Deleting alias 55BQIJMVGI from agent WJYFKTH1IX
Deleting alias TSTALIASID from agent WJYFKTH1IX
Deleting agent: WJYFKTH1IX...
Deleting IAM role: AmazonBedrockExecutionRoleForAgents_dsl-query-agent-us-west-2-533...
Creating agent dsl-query-agent-us-west-2-533...
Created agent, id: AK3UI7V9RL, alias id: TSTALIASID

Adding action group with Lambda: arn:aws:lambda:us-west-2:533267284022:function:execute-dsl-query-us-west-2-533...
Waiting for agent status to change. Current status CREATING
Agent id AK3UI7V9RL current status: NOT_PREPARED
Waiting for agent status to change. Current status VERSIONING
Agent id AK3UI7V9RL current status: PREPARED
DONE: Agent: dsl-query-agent-us-west-2-533, id: AK3UI7V9RL, alias id: BDZDCS2YJE



In [10]:
dsl_query_agent_id = agents_helper.get_agent_id_by_name(f"dsl-query-agent-{agent_suffix}")
print(f"Agent ID for DSL Query Agent: {dsl_query_agent_id}")

Agent ID for DSL Query Agent: AK3UI7V9RL


In [25]:
# Define function and role names
LAMBDA_FUNCTION_NAME = f"execute-dsl-query-{agent_suffix}"
IAM_ROLE_NAME = f"LambdaExecutionRole-{agent_suffix}"
ZIP_FILE_NAME = "function.zip"
LAMBDA_FILE_PATH = "src/lambda/execute_dsl_query.py"
DEPENDENCIES = ["opensearch-py", "requests", "urllib3"]  # Required packages

def create_iam_role(role_name):
    """Creates an IAM Role with necessary trust policy for Lambda."""
    
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    try:
        role = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
        )
        print(f"IAM Role {role_name} created.")
    except iam_client.exceptions.EntityAlreadyExistsException:
        print(f"IAM Role {role_name} already exists.")
        role = iam_client.get_role(RoleName=role_name)

    # Attach AWS Lambda execution policy
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn="arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole"
    )



    # Attach additional policies for OpenSearch access
    opensearch_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "es:Describe*",
                    "es:List*",
                    "es:Get*"
                ],
                "Resource": "*"
            }
        ]
    }

    opensearch_policy_name = f"{role_name}-OpenSearchPolicy"
    try:
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName=opensearch_policy_name,
            PolicyDocument=json.dumps(opensearch_policy_document)
        )
        print(f"Attached OpenSearch policy to IAM Role {role_name}.")
    except Exception as e:
        print(f"Failed to attach OpenSearch policy to IAM Role {role_name}: {e}")


    # Attach the new policy for aoss:APICall
    aoss_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "aoss:*"
                ],
                "Resource": "*"
            }
        ]
    }

    aoss_policy_name = f"{role_name}-AOSSPolicy"
    try:
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName=aoss_policy_name,
            PolicyDocument=json.dumps(aoss_policy_document)
        )
        print(f"Attached AOSS policy to IAM Role {role_name}.")
    except Exception as e:
        print(f"Failed to attach AOSS policy to IAM Role {role_name}: {e}")


    role_arn = role['Role']['Arn']

    # Wait for IAM role to propagate
    print("Waiting for IAM role to be usable...")
    time.sleep(10)  # Delay to allow propagation

    return role_arn


def create_zip_file(source_file, zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'w') as zipf:
        zipf.write(source_file, os.path.basename(source_file))

def create_lambda_package(source_file, zip_file_path):
    """Packages the Lambda function and its dependencies into a ZIP file."""
    package_dir = "package"
    
    # Install dependencies
    if not os.path.exists(package_dir):
        os.makedirs(package_dir)
    print("Installing dependencies...")
    subprocess.run(f"pip install {' '.join(DEPENDENCIES)} -t {package_dir}", shell=True, check=True)

    # Create ZIP file with dependencies and function
    print("Creating Lambda deployment package...")
    with zipfile.ZipFile(zip_file_path, 'w') as zipf:
        for root, _, files in os.walk(package_dir):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, package_dir))

        zipf.write(source_file, os.path.basename(source_file))

    # Cleanup temporary package directory
    subprocess.run(f"rm -rf {package_dir}", shell=True)
    print("Lambda package created successfully.")


def create_lambda_function(function_name, role_arn, handler, runtime, zip_file_path, region_name=region):
    # Initialize a session using Amazon Lambda
    lambda_client = boto3.client('lambda', region_name=region_name)

    # Read the zip file content
    with open(zip_file_path, 'rb') as f:
        zip_content = f.read()

    # Create the Lambda function
    try:
        response = lambda_client.create_function(
            FunctionName=function_name,
            Runtime=runtime,
            Role=role_arn,
            Handler=handler,
            Code={'ZipFile': zip_content},
            Description='Lambda function to execute DSL queries',
            Timeout=15,
            MemorySize=128,
            Publish=True
        )
        print(f"Lambda function {function_name} created successfully.")
    except lambda_client.exceptions.ResourceConflictException:
        print(f"Lambda function {function_name} already exists. Updating the function...")

        response = lambda_client.update_function_code(
            FunctionName=function_name,
            ZipFile=zip_content
        )
        print(f"Lambda function {function_name} updated successfully.")

    return response

def add_resource_based_policy(function_name, agent_id, region, account_id):
    statement_id = "AllowExecutionFromBedrockAgent"
    policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": statement_id,
                "Effect": "Allow",
                "Principal": {
                    "Service": "bedrock.amazonaws.com"
                },
                "Action": "lambda:InvokeFunction",
                "Resource": f"arn:aws:lambda:{region}:{account_id}:function:{function_name}",
                "Condition": {
                    "ArnLike": {
                        "AWS:SourceArn": f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}"
                    }
                }
            }
        ]
    }

    # Check if the statement ID already exists and remove it if necessary
    try:
        existing_policy = lambda_client.get_policy(FunctionName=function_name)
        policy_document = json.loads(existing_policy['Policy'])
        for statement in policy_document['Statement']:
            if statement['Sid'] == statement_id:
                lambda_client.remove_permission(
                    FunctionName=function_name,
                    StatementId=statement_id
                )
                print(f"Removed existing statement ID {statement_id} from Lambda function {function_name}.")
                break
    except lambda_client.exceptions.ResourceNotFoundException:
        print(f"No existing policy found for Lambda function {function_name}.")
    except Exception as e:
        print(f"Error checking existing policy for Lambda function {function_name}: {e}")

    # Add the new permission
    try:
        response = lambda_client.add_permission(
            FunctionName=function_name,
            StatementId=statement_id,
            Action=policy['Statement'][0]['Action'],
            Principal=policy['Statement'][0]['Principal']['Service'],
            SourceArn=policy['Statement'][0]['Condition']['ArnLike']['AWS:SourceArn']
        )
        print(f"Resource-based policy added to Lambda function {function_name}.")
    except Exception as e:
        print(f"Failed to add resource-based policy to Lambda function {function_name}: {e}")



In [26]:

# Step 1: Create IAM Role for Lambda
role_arn = create_iam_role(IAM_ROLE_NAME)

# Step 2: Zip the existing lambda function file
if not os.path.exists(LAMBDA_FILE_PATH):
    print(f"Error: {LAMBDA_FILE_PATH} does not exist. Create the file first.")
    exit(1)

# Step 3: Package Lambda function and dependencies
create_lambda_package(LAMBDA_FILE_PATH, ZIP_FILE_NAME)

# Step 4: Create or update the Lambda function
response = create_lambda_function(
    function_name=LAMBDA_FUNCTION_NAME,
    role_arn=role_arn,
    handler="execute_dsl_query.lambda_handler",
    runtime="python3.12",
    zip_file_path=ZIP_FILE_NAME
)

# Step 4: Add resource-based policy to the Lambda function
add_resource_based_policy(LAMBDA_FUNCTION_NAME, dsl_query_agent_id, region, account_id)


# Cleanup: Remove temporary ZIP file
os.remove(ZIP_FILE_NAME)
print(response)

IAM Role LambdaExecutionRole-us-west-2-533 already exists.
Attached OpenSearch policy to IAM Role LambdaExecutionRole-us-west-2-533.
Attached AOSS policy to IAM Role LambdaExecutionRole-us-west-2-533.
Waiting for IAM role to be usable...
Installing dependencies...
  Using cached opensearch_py-2.8.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached Events-0.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-macosx_10_13_universal2.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached opensearch_py-2.8.0-py3-none-any.whl (353 kB)
Using cached requests-2.32.3-py3-none-any.wh

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.16.1 requires botocore<1.35.89,>=1.35.74, but you have botocore 1.36.9 which is incompatible.
datasets 2.21.0 requires dill<0.3.9,>=0.3.0, but you have dill 0.3.9 which is incompatible.
datasets 2.21.0 requires fsspec[http]<=2024.6.1,>=2023.1.0, but you have fsspec 2024.12.0 which is incompatible.
awscli 1.34.8 requires botocore==1.35.8, but you have botocore 1.36.9 which is incompatible.
awscli 1.34.8 requires s3transfer<0.11.0,>=0.10.0, but you have s3transfer 0.11.2 which is incompatible.


Creating Lambda deployment package...
Lambda package created successfully.
Lambda function execute-dsl-query-us-west-2-533 already exists. Updating the function...
Lambda function execute-dsl-query-us-west-2-533 updated successfully.
Removed existing statement ID AllowExecutionFromBedrockAgent from Lambda function execute-dsl-query-us-west-2-533.
Resource-based policy added to Lambda function execute-dsl-query-us-west-2-533.
{'ResponseMetadata': {'RequestId': '280bceeb-4d56-4d93-878e-bf1865ed8138', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 01 Feb 2025 19:05:17 GMT', 'content-type': 'application/json', 'content-length': '1449', 'connection': 'keep-alive', 'x-amzn-requestid': '280bceeb-4d56-4d93-878e-bf1865ed8138'}, 'RetryAttempts': 0}, 'FunctionName': 'execute-dsl-query-us-west-2-533', 'FunctionArn': 'arn:aws:lambda:us-west-2:533267284022:function:execute-dsl-query-us-west-2-533', 'Runtime': 'python3.12', 'Role': 'arn:aws:iam::533267284022:role/LambdaExecutionRole-us-west-2-5